In [2]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

Спарсим данные о временах в квалификациях с сайта, с которого мы парсили погоду в гонках, информации про которые не было в википедии. На этом сайте информация про каждую гонку находится по ссылкам такого вида, причем в ссылках для каждой гонки вся разница заключается только в последнем числе. Таким образом, гонки достаточно легко перебираются.
https://f1-data.com/script/db/race/classify.php?race=1

Мы будем парсить информацию для всех гонок, потому что пропусков во временах квалификаций значительно больше, чем пропусков в колонке с погодой, поэтому можно просто перебрать все гонки, а не искать только те, в которых нет информации

In [38]:
all_links = []
numbers = []
times = []
race_newid = []
for i in range(1, 1085):
    all_links.append('http://f1-data.com/script/db/race/classify.php?race=' + str(i))
    html = all_links[-1]
    response = requests.get(url=all_links[-1])    
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table')
    names = soup.find_all('h2')
    quali_table = []
    for k in range(0, len(table)-1):
            if (str(names[k]).find('Qualify') != -1):
                    quali_table.append(table[k])
    tr = quali_table[0].find_all('tr')
    for j in range(1, len(tr)): #Будем перебирать каждого гонщика
        td = tr[j].find_all('td')
        driver_no = str(td[1]).split('>')[1].split('<')[0]
        driver_time = str(td[4]).split('>')[1].split('<')[0]
        numbers.append(driver_no) # список, в котором хранятся номера гонщиков (не driverId, a number, как в таблице quali)
        times.append(driver_time) # список, в котором хранятся времена квалификаций (мы брали их из итогового протокола квалификации, а не лучшее время по сегментов, чтобы не перебирать большое количество случаев)
        race_newid.append(i) # список, в котором хранятся newid гонок
        if i == 1 and j == 1:
            time_quali = pd.DataFrame({'raceNewid': race_newid, 'driverNumber': numbers, 'driverTime': times})
            
        else:
            time_quali.loc[len(time_quali.index)] = [i, driver_no, driver_time]

Посмотрим, что у нас получилось

In [39]:
time_quali.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25457 entries, 0 to 25456
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceNewid     25457 non-null  int64 
 1   driverNumber  25457 non-null  object
 2   driverTime    25457 non-null  object
dtypes: int64(1), object(2)
memory usage: 795.5+ KB


На самом деле, в этой таблице есть пропуски, так как периодически гонщики могут не участвовать в квалификации, например, из-за того, что у них сломалась машина и команда не успевает починить ее до начала квалификации.

In [40]:
#Используем функции, написанные нами в основной части для того, чтобы перевести время в миллисекунлы
def millisec(x):
    if len(x) == 3:
        min_ = int(x[0]) * 60000
        sec_ = int(x[1]) * 1000
        millis_ = int(x[2])
        y = min_ + sec_ + millis_
    else:
        y = 0
    return y

timing = np.vectorize(millisec)

def separating(qual):
    time = np.array(time_quali[qual].fillna('no_time'))
    time = time.astype(str)
    time = np.char.replace(time, '.', ':')
    time = np.char.split(time, sep =':')
    time = timing(time)
    return(time)

In [41]:
time_quali['driverTimeMill'] = separating('driverTime')

Запишем эту таблицу в файл и сохраним его на github, чтобы потом на него ссылаться

Посмотрим теперь на количество пропусков (в таких строках driverTimeMill = 0)

In [45]:
time_quali.loc[time_quali['driverTimeMill'] == 0]

,raceNewid,driverNumber,driverTime,driverTimeMill
38,2,4,,0
40,2,8,,0
41,2,28,,0
43,3,28,,0
44,3,31,,0
...,...,...,...,...
24956,1059,22,,0
24976,1060,18,,0
24996,1061,23,,0
25016,1062,31,,0


In [129]:
time_quali.loc[time_quali['raceNewid'] == 1083]

,raceNewid,driverNumber,driverTime,driverTimeMill
25368,1083,,11,0


In [31]:
time_quali = pd.read_csv('quali_time_full.csv')
time_quali

,Unnamed: 0,raceNewid,driverNumber,driverTime,driverTimeMill
0,0,1,2,1:50.800,110800
1,1,1,3,1:51.000,111000
2,2,1,1,1:51.000,111000
3,3,1,4,1:52.200,112200
4,4,1,21,1:52.600,112600
...,...,...,...,...,...
25452,25452,1084,4,1:28.394,88394
25453,25453,1084,22,1:28.429,88429
25454,25454,1084,18,1:28.476,88476
25455,25455,1084,81,1:28.484,88484


In [127]:
a = list(time_quali['raceNewid'].unique())

In [128]:
a

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [56]:
quali = pd.read_csv('https://raw.githubusercontent.com/pavleke/project-formula-1/main/qualifying.csv')
races = pd.read_csv('https://raw.githubusercontent.com/pavleke/project-formula-1/main/races.csv')
result = pd.read_csv('https://raw.githubusercontent.com/pavleke/project-formula-1/main/results.csv')

In [20]:
new = races.sort_values(by='date')
new['newid'] = np.arange(1, len(new)+1)
new = new[['raceId', 'newid']]

In [57]:
result

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25841,1096,854,210,47,12,16,16,16,0.0,57,\N,\N,39,12,1:29.833,211.632,11
25836,25842,1096,825,210,20,16,17,17,17,0.0,57,\N,\N,40,20,1:31.158,208.556,11
25837,25843,1096,1,131,44,5,18,18,18,0.0,55,\N,\N,42,11,1:29.788,211.738,9
25838,25844,1096,849,3,6,20,19,19,19,0.0,55,\N,\N,45,14,1:30.309,210.517,130


In [61]:
result = result.merge(new, how='left', on='raceId')

In [63]:
result.rename(columns={"newid": "raceNewid", "number": "driverNumber"}, inplace=True)

In [62]:
result

,resultId,raceId,driverId,constructorId,driverNumber,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,newid
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,786
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,786
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,786
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,786
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25841,1096,854,210,47,12,16,16,16,0.0,57,\N,\N,39,12,1:29.833,211.632,11,1079
25836,25842,1096,825,210,20,16,17,17,17,0.0,57,\N,\N,40,20,1:31.158,208.556,11,1079
25837,25843,1096,1,131,44,5,18,18,18,0.0,55,\N,\N,42,11,1:29.788,211.738,9,1079
25838,25844,1096,849,3,6,20,19,19,19,0.0,55,\N,\N,45,14,1:30.309,210.517,130,1079


In [95]:
result.reset_index(inplace=True)

In [96]:
result

,raceNewid,driverNumber,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,786,22,1,18,1,1,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,786,3,2,18,2,2,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,786,7,3,18,3,3,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,786,5,4,18,4,4,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,786,23,5,18,5,1,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1079,47,25841,1096,854,210,12,16,16,16,0.0,57,\N,\N,39,12,1:29.833,211.632,11
25836,1079,20,25842,1096,825,210,16,17,17,17,0.0,57,\N,\N,40,20,1:31.158,208.556,11
25837,1079,44,25843,1096,1,131,5,18,18,18,0.0,55,\N,\N,42,11,1:29.788,211.738,9
25838,1079,6,25844,1096,849,3,20,19,19,19,0.0,55,\N,\N,45,14,1:30.309,210.517,130


In [97]:
time_quali.reset_index(inplace=True)

In [115]:
result["driverNumber"].replace('\\N', 0, inplace=True)
result["driverNumber"] = result['driverNumber'].astype('int')

In [116]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25840 entries, 0 to 25839
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   raceNewid        25840 non-null  int64  
 1   driverNumber     25840 non-null  int32  
 2   resultId         25840 non-null  int64  
 3   raceId           25840 non-null  int64  
 4   driverId         25840 non-null  int64  
 5   constructorId    25840 non-null  int64  
 6   grid             25840 non-null  int64  
 7   position         25840 non-null  object 
 8   positionText     25840 non-null  object 
 9   positionOrder    25840 non-null  int64  
 10  points           25840 non-null  float64
 11  laps             25840 non-null  int64  
 12  time             25840 non-null  object 
 13  milliseconds     25840 non-null  object 
 14  fastestLap       25840 non-null  object 
 15  rank             25840 non-null  object 
 16  fastestLapTime   25840 non-null  object 
 17  fastestLapSp

In [118]:
merged = result.merge(time_quali, how='outer', on=['raceNewid', 'driverNumber'])

In [129]:
merged

,raceNewid,driverNumber,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,...,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,Unnamed: 0,driverTime,driverTimeMill
0,786,22,1.0,18.0,1.0,1.0,1.0,1,1,1.0,...,1:34:50.616,5690616,39,2,1:27.452,218.300,1.0,19139.0,1:26.572,86572.0
1,786,3,2.0,18.0,2.0,2.0,5.0,2,2,2.0,...,+5.478,5696094,41,3,1:27.739,217.586,1.0,19131.0,1:25.960,85960.0
2,786,7,3.0,18.0,3.0,3.0,7.0,3,3,3.0,...,+8.163,5698779,41,5,1:28.090,216.719,1.0,19135.0,1:26.295,86295.0
3,786,5,4.0,18.0,4.0,4.0,11.0,4,4,4.0,...,+17.181,5707797,58,7,1:28.603,215.464,1.0,19143.0,1:26.907,86907.0
4,786,23,5.0,18.0,5.0,1.0,3.0,5,5,5.0,...,+18.014,5708630,43,1,1:27.418,218.385,1.0,19130.0,1:25.664,85664.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26125,1084,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25452.0,1:28.394,88394.0
26126,1084,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25453.0,1:28.429,88429.0
26127,1084,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25454.0,1:28.476,88476.0
26128,1084,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25455.0,1:28.484,88484.0


In [134]:
quali_full = merged[['raceNewid', 'driverNumber', 'resultId', 'raceId', 'driverId', 'driverTime', 'driverTimeMill']]

In [135]:
quali_full.to_csv('quali_full.csv')